Instituto Politécnico Nacional

Centro de Investigación en Computación

Minería de datos I

Análisis exploratorio de datos de trayectorias de albatros de Laysan

Presenta: Ana Maritza Bello

---

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Visualización de datos crudos

In [ ]:
trajectories = pd.read_csv('../data/trajectories.csv')
trajectories.head()

# Tipos de datos en el dataset
Se tiene un total de 176767 puntos de ocurrencias de 47 aves marinas obtenidos
con GPS. Los datos son del año 2014 a 2018. Las variables con las que se cuentan
son la fecha de la ocurrencia, las coordenadas de latitud y longitud, el nombre
del individuo, la temporada reproductiva de la especie, la distancia de la
ocurrencia a la zona de anidamiento más cercana (Isla Guadalupe) y el valor de
la temperatura superficial del mar en el punto de ocurrencia.

Los datos de tipo object son cadenas de caracteres. La columna date es de tipo
datetime64[ns] y las demás son de tipo float64.


In [ ]:
trajectories["date"] = trajectories["date"].astype("datetime64[ns]")
trajectories.info()

# Cantidad de valores únicos en cada variable del dataset

Se tiene un total de 346 fechas registradas, 47 individuos, 4 temporadas
reproductivas y 5 años de registro. Las variables de latitud, longitud,
distancia y temperatura superficial del mar son medidas muy variadas, por esto
su cantidad de valores únicos es muy alta.

In [ ]:
trajectories.nunique()

# Gráfica de la cantidad de datos disponibles por año y por mes

Saber la cantidad de puntos que se tienen por mes y por año es importante para
saber si se tienen suficientes datos para hacer un análisis por mes o por año.
En este caso, la cantidad de datos por mes es muy variable, por lo que si se
desea hacer un análisis de datos por mes, solo se tendría información disponible
de los primeros 3 o 4 meses dependiendo del año.

In [ ]:
trajectories["day"] = trajectories["date"].dt.day
trajectories["month"] = trajectories["date"].dt.month
trajectories["year"] = trajectories["date"].dt.year
data_by_year = trajectories.groupby(["year", "month"]).size()
data_by_year.plot(kind="bar", figsize=(10, 5), 
                  title="Numero de datos por mes y año", 
                  xlabel="Año, mes", ylabel="Cantidad de datos")

In [ ]:
data_by_month = trajectories.groupby(["month"]).size()
data_by_month.plot(kind="bar", figsize=(10, 5), 
                  title="Numero de datos por mes (todos los años)", 
                  xlabel="Mes", ylabel="Cantidad de datos",)

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(data=trajectories, x="season", palette="Set1")
plt.title("Cantidad de datos disponibles por temporada reproductiva de todos los años")
plt.xlabel("Temporada reproductiva")
plt.ylabel("Cantidad de datos")

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(data=trajectories, x="season", hue="year", palette="Set2",)
plt.title("Cantidad de datos disponibles por temporada reproductiva y año")
plt.xlabel("Temporada reproductiva")
plt.ylabel("Cantidad de datos")

In [ ]:
sst_eda = trajectories["sst"].groupby(trajectories["month"])
sst_eda.describe()

# Gráficas del valor de la temperatura superficial del mar.

Estas gráficas muestran la distribución de los valores de la temperatura del mar
en los puntos de ocurrencia de las aves. Se puede observar que la preferencia de
estas aves es por temperaturas que vand desde los 5 grados centígrados
aproximadamente hasta los 25 grados centígrados.

In [ ]:
# Plot EDA for SST (Min, mean, max)

plt.figure(figsize=(10, 5))
plt.title("Temperatura superficial del mar (SST) de los puntos de las trayectorias por mes")
sst_eda.mean().plot(label="Mean", color="green", linewidth=2, marker="o")
sst_eda.min().plot(label="Min", color="blue", linewidth=2, marker="o")
sst_eda.max().plot(label="Max", color='red', linewidth=2, marker="o")
plt.legend()
plt.xlabel("Mes")
plt.ylabel("SST (°C)")


In [ ]:
# Boxplots for SST by month

plt.figure(figsize=(12, 5))
plt.title("Temperatura superficial del mar (SST) de los puntos de las trayectorias por mes")
sns.boxplot(data=trajectories, x="month", y="sst", palette="Set2")
plt.xlabel("Mes")
plt.ylabel("SST (°C)")


# Distancia de los puntos de ocurrencia a la zona de anidamiento más cercana

In [ ]:
distance_eda = trajectories["spheroid_dist_to_colony"].groupby(trajectories["month"])
distance_eda.describe()

In [ ]:
# Gráfica de la distancia a la colonia por mes (Min, mean, max)

plt.figure(figsize=(10, 5))
plt.title("Distancia máxima a la colonia por mes")
distance_eda.max().plot(label="Max")
plt.legend()
plt.xlabel("Mes")
plt.ylabel("Distancia (km)")


# Análisis exploratorio de datos de los valores de la longitud de las ocurrencias por temporada reproductiva

Este análisis nos siver para saber que tanto varía el área de distribución de
las aves por temporada reproductiva. Así, podemos saber la preferencia de
dispersión en longitud de las aves por temporada reproductiva.

In [ ]:
latitude_bp = trajectories["longitude"].groupby(trajectories["season"])
latitude_bp.describe()

# Análisis exploratorio de datos de los valores de la latitud de las ocurrencias por temporada reproductiva

Este análisis nos siver para saber que tanto varía el área de distribución de las aves por temporada reproductiva. Así, podemos saber la preferencia de dispersión en latitud de las aves por temporada reproductiva.

In [ ]:
latitude_bp = trajectories["latitude"].groupby(trajectories["season"])
latitude_bp.describe()

# Dispersión de los puntos de ocurrencia por temporada reproductiva

La dispersión de los puntos de ocurrencia por temporada reproductiva en el
espacio geográfico nos permite saber si las aves se distribuyen de manera
similar en las diferentes temporadas reproductivas. También nos permite
determinar zonas con mayor densidad de ocurrencias, esto es útil para
identificar áreas de importancia para las aves.

In [ ]:
sns.scatterplot(data = trajectories, x="longitude", y="latitude", hue="season", alpha=0.5, s=10)

# Distribución geográfica durante la incubación

In [ ]:
sns.boxplot(data = trajectories[(trajectories["season"]=="incubacion") & (trajectories["year"]==2014)], x="longitude")
plt.title("Boxplot de la longitud en la temporada de incubacion del año 2014")


In [ ]:
sns.boxplot(data = trajectories[(trajectories["season"]=="incubacion") & (trajectories["year"]==2015)], x="longitude")
plt.title("Boxplot de la longitud en la temporada de incubacion del año 2015")

In [ ]:
sns.histplot(data = trajectories[(trajectories["season"]=="incubacion") & (trajectories["year"]==2014)],
             x="longitude", binwidth=0.5, kde=True, color="red", alpha=0.3)
plt.title("Histograma de la longitud en la temporada de incubacion del año 2014")

In [ ]:
sns.boxplot(data = trajectories[(trajectories["season"]=="incubacion") & (trajectories["year"]==2014)], x="latitude")
plt.title("Boxplot de la latitud en la temporada de incubacion del año 2014")

In [ ]:
sns.boxplot(data = trajectories[(trajectories["season"]=="incubacion") & (trajectories["year"]==2015)], x="latitude")
plt.title("Boxplot de la latitud en la temporada de incubacion del año 2015")

In [ ]:
sns.histplot(data = trajectories[(trajectories["season"]=="incubacion") & (trajectories["year"]==2014)],
             x="latitude", binwidth=0.5, kde=True, color="green", alpha=0.2)
plt.title("Histograma de la latitud en la temporada de incubacion")

# Distribución geográfica durante la crianza

In [ ]:
sns.boxplot(data = trajectories[trajectories["season"]=="crianza"], x="longitude")
plt.title("Boxplot de la longitud en la temporada de crianza")

In [ ]:
sns.histplot(data = trajectories[trajectories["season"]=="crianza"],
             x="longitude", binwidth=0.5, kde=True, color="red", alpha=0.3)
plt.title("Histograma de la longitud en la temporada de crianza")

In [ ]:
sns.boxplot(data = trajectories[trajectories["season"]=="crianza"], x="latitude")
plt.title("Boxplot de la latitud en la temporada de crianza")

In [ ]:
sns.histplot(data = trajectories[trajectories["season"]=="crianza"],
             x="latitude", binwidth=0.5, kde=True, color="green", alpha=0.3)
plt.title("Histograma de la latitud en la temporada de crianza")

# Correlación entre las variables

La correlación entre las variables nos permite saber si existe una relación
entre ellas. El signo de la correlación nos indica si la relación es directa o
inversa. El valor de la correlación nos indica que tan fuerte es la relación que 
existe entre las variables.

In [ ]:
data_trajectories = trajectories[["longitude", "latitude", "sst", "spheroid_dist_to_colony", "year", "month"]]
sns.heatmap(data_trajectories.corr(), annot=True, cmap="viridis")

# Pairplot de las variables

El pairplot nos permite ver la relación entre las variables de dos en dos.

In [ ]:
sns.pairplot(data_trajectories, hue="year", palette="Set2")